<h1>UK Road Safety Analysis</h1>

<p>This notebook analyzes road accident data in the UK. The goal is to identify trends and patterns to improve road safety.</p>

<h2>Data Overview</h2>
<p>The dataset contains information about road accidents in the UK, including:</p>
<ul>
  <li>Accident details (e.g. time, severity, location).</li>
  <li>Environmental conditions (e.g. weather, lighting, road surface).</li>
  <li>Junction and road characteristics (e.g. junction type, speed limit, road type).</li>
  <li>The number of casualties.</li>
</ul>

<h2>Analysis</h2>
<ol>
  <li>Data Cleaning</li>
  <li>Exploratory Data Analysis (EDA)</li>
  <li>Visualization</li>
  <li>Conclusion</li>
</ol>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h2>Loading the Data, Cleaning, and Formatting</h2>
<p>First, we load the data using Pandas and use <code>.head()</code> to preview the dataset. This allows us to see the structure of the data and decide which columns will be most useful for generating insights.</p>


In [2]:
data = pd.read_csv('Road Accident Data.csv')

data.head()

,Accident_Index,Accident Date,Day_of_Week,Junction_Control,Junction_Detail,Accident_Severity,Latitude,Light_Conditions,Local_Authority_(District),Carriageway_Hazards,...,Number_of_Casualties,Number_of_Vehicles,Police_Force,Road_Surface_Conditions,Road_Type,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions,Vehicle_Type
0,BS0000001,01-01-2021,Thursday,Give way or uncontrolled,T or staggered junction,Serious,51.512273,Daylight,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Dry,One way street,30,15:11,Urban,Fine no high winds,Car
1,BS0000002,05-01-2021,Monday,Give way or uncontrolled,Crossroads,Serious,51.514399,Daylight,Kensington and Chelsea,NaN,...,11,2,Metropolitan Police,Wet or damp,Single carriageway,30,10:59,Urban,Fine no high winds,Taxi/Private hire car
2,BS0000003,04-01-2021,Sunday,Give way or uncontrolled,T or staggered junction,Slight,51.486668,Daylight,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Dry,Single carriageway,30,14:19,Urban,Fine no high winds,Taxi/Private hire car
3,BS0000004,05-01-2021,Monday,Auto traffic signal,T or staggered junction,Serious,51.507804,Daylight,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Frost or ice,Single carriageway,30,08:10,Urban,Other,Motorcycle over 500cc
4,BS0000005,06-01-2021,Tuesday,Auto traffic signal,Crossroads,Serious,51.482076,Darkness - lights lit,Kensington and Chelsea,NaN,...,1,2,Metropolitan Police,Dry,Single carriageway,30,17:25,Urban,Fine no high winds,Car


<p>Next, we will remove all the columns that we won’t be using by applying the <code>.drop()</code> method to the DataFrame. After dropping these columns, we use <code>.info()</code> to check which columns remain and to verify whether there is any missing data in the dataset.</p>

In [3]:
data.drop(['Carriageway_Hazards', 'Police_Force', 'Vehicle_Type', 'Accident Date', 'Accident_Index', 'Latitude', 'Longitude'], axis=1, inplace=True, errors='ignore')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307973 entries, 0 to 307972
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   Day_of_Week                 307973 non-null  object
 1   Junction_Control            307973 non-null  object
 2   Junction_Detail             307973 non-null  object
 3   Accident_Severity           307973 non-null  object
 4   Light_Conditions            307973 non-null  object
 5   Local_Authority_(District)  307973 non-null  object
 6   Number_of_Casualties        307973 non-null  int64 
 7   Number_of_Vehicles          307973 non-null  int64 
 8   Road_Surface_Conditions     307973 non-null  object
 9   Road_Type                   307973 non-null  object
 10  Speed_limit                 307973 non-null  int64 
 11  Time                        307956 non-null  object
 12  Urban_or_Rural_Area         307973 non-null  object
 13  Weather_Conditions          3

<p>Before we start exploring the data, we convert the Time column into a 64-bit datetime object. This allows us to perform various calculations when analyzing the dataset. For example, converting the column enables us to calculate the median time, which would not be possible if it were stored as plain strings.</p>
<p>We then use this median value to fill in any missing Time entries that were identified in the previous step. Finally, we verify the changes using .<code>info()</code> and <code>.head()</code> to confirm the column was updated correctly.</p>
<p><bold>Note: All times now display with the date 1900-01-01. This is simply a placeholder added by Pandas when creating a full datetime object. It does not affect calculations or the accuracy of the time values</bold></p>

In [7]:
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M', errors='coerce')

median_time = data['Time'].median()
data['Time'] = data['Time'].fillna(median_time)

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307973 entries, 0 to 307972
Data columns (total 14 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Day_of_Week                 307973 non-null  object        
 1   Junction_Control            307973 non-null  object        
 2   Junction_Detail             307973 non-null  object        
 3   Accident_Severity           307973 non-null  object        
 4   Light_Conditions            307973 non-null  object        
 5   Local_Authority_(District)  307973 non-null  object        
 6   Number_of_Casualties        307973 non-null  int64         
 7   Number_of_Vehicles          307973 non-null  int64         
 8   Road_Surface_Conditions     307973 non-null  object        
 9   Road_Type                   307973 non-null  object        
 10  Speed_limit                 307973 non-null  int64         
 11  Time                        307973 non-

,Day_of_Week,Junction_Control,Junction_Detail,Accident_Severity,Light_Conditions,Local_Authority_(District),Number_of_Casualties,Number_of_Vehicles,Road_Surface_Conditions,Road_Type,Speed_limit,Time,Urban_or_Rural_Area,Weather_Conditions
0,Thursday,Give way or uncontrolled,T or staggered junction,Serious,Daylight,Kensington and Chelsea,1,2,Dry,One way street,30,1900-01-01 15:11:00,Urban,Fine no high winds
1,Monday,Give way or uncontrolled,Crossroads,Serious,Daylight,Kensington and Chelsea,11,2,Wet or damp,Single carriageway,30,1900-01-01 10:59:00,Urban,Fine no high winds
2,Sunday,Give way or uncontrolled,T or staggered junction,Slight,Daylight,Kensington and Chelsea,1,2,Dry,Single carriageway,30,1900-01-01 14:19:00,Urban,Fine no high winds
3,Monday,Auto traffic signal,T or staggered junction,Serious,Daylight,Kensington and Chelsea,1,2,Frost or ice,Single carriageway,30,1900-01-01 08:10:00,Urban,Other
4,Tuesday,Auto traffic signal,Crossroads,Serious,Darkness - lights lit,Kensington and Chelsea,1,2,Dry,Single carriageway,30,1900-01-01 17:25:00,Urban,Fine no high winds


<h2></h2>